# Hensel's Lemma

In this python3 notebook an implementation of a variation of Hensel's lemma is presented.

In [1]:
import sympy
import math

In [2]:
sympy.init_printing(use_latex=True)

To implement it, we require the following lemma:

**Lemma 1**: let $p(y)$ and $q(y)$ be polynomials in $\mathbb{K}[y]$ with $\gcd(p,q) = 1$, $\deg(p) = r$ and $\deg(q) = s$. Given $f\in\mathbb{K}[y]$ such that $\deg(f) < r+s$, there exists unique polynomials $g,h\in\mathbb{K}[y]$ such that $f = gp+hq$

The implementation of this lema can be found here:

In [3]:
infinity = sympy.simplify('oo')

In [4]:
infinity

In [5]:
3 > infinity

In [6]:
def lemma_1(f,p,q):
    r = sympy.degree(p)
    s = sympy.degree(q)
    if sympy.degree(f) >= r+s:
        raise ValueError('polynomial f must have degree less that deg(p)+deg(q)')
    (phi, psi, gcd) = sympy.gcdex(p,q)
    if gcd != 1:
        raise ValueError('polynomials p and q are not relatively prime')
    l, h = sympy.div(f*psi, p)
    g = f*phi + l*q
    return g, h

Next, we require two auxiliar functions that helps us deal with factorizations and homogenous components:

In [7]:
def fgetter(F):
    '''
    This functions gets the homogenous components with respect to x. They are interpreted as polynomials in y.
    
    F: a sympy expression in two variables x and y.
    returns: a list of homogenous components.
    
    TO-DO:
        - find a way to get the generators of a polynomial, so that I can implement this in a more general fashion.
    '''
    x = sympy.Symbol('x')
    y = sympy.Symbol('y')
    list_of_Fprimes = [F]
    list_of_fs = [F.subs(x, 0)]
    for j in range(1, sympy.degree(F, x)+1):
        nextFprime = sympy.diff(list_of_Fprimes[j-1], x)
        list_of_Fprimes.append(nextFprime)
        list_of_fs.append(sympy.Rational(1, math.factorial(j))*nextFprime.subs(x, 0))
    return list_of_fs

In [8]:
def pqgetter(f):
    '''
    This function takes a polynomial f = pq with gcd(p,q) = 1 and returns the tuple (p, q).
    '''
    if f.is_irreducible:
        raise ValueError('f has no factorization')
    _list_of_factors = list(sympy.Mul.make_args(f.factor()))
    if len(_list_of_factors) < 2:
        raise ValueError('f might have no factorization f=pq with gcd(p,q) = 1')
    if len(_list_of_factors) == 2:
        if sympy.gcd(_list_of_factors[0], _list_of_factors[1]) == 1:
            return _list_of_factors[0], _list_of_factors[1]
        else:
            raise ValueError('f has no factorization f=pq with gcd(p,q) = 1')
    else:
        first_factor = _list_of_factors[0]
        second_factor = 1
        for index in range(1, len(_list_of_factors)):
            second_factor = second_factor * _list_of_factors[index]
        if sympy.gcd(first_factor, second_factor) == 1:
            return first_factor, second_factor
        else:
            raise ValueError('f has no factorization f=pq with gcd(p,q) = 1')

Now we implement a version of Hensel's lemma:

**Lemma (Hensel)**: Let $F\in \mathbb{K}[[x]][y]$ be a monic polynomial such that $F(0,y) = p(y)q(y)$ with $\gcd(p,q) = 1$, $\deg(p) = r$ and $\deg(q) = s$. Then there exists polynomials $P,Q\in\mathbb{K}[[x]][y]$ with $\deg(P) = r$ and $\deg(Q) = s$ such that $P(0,y) = p(y)$, $Q(0,y) = q(y)$ and $F = PQ$.

In [9]:
def HenselsLemma(F):
    '''
    This function takes a polynomial in two variables F(x,y) monic in y such that F(0,y) = p(y)q(y), and finds
    two polynomials in two variables P and Q such that P(0,y) = p(y), Q(0,y) = q(y) and F = PQ up to deg(F).
    That is, this function lifts the factorization F(0,y) = p(y)q(y).
    
    F: a polynomial in two variables x and y.
    returns: a tuple (P,Q) such that F = PQ up to deg(F).
    
    TO-DO:
        - this function only accepts polynomials in explicitly x and y, we need one that accepts polys in 
          arbitrary generators.
    '''
    x = sympy.Symbol('x')
    y = sympy.Symbol('y')
    if sympy.LC(F,y) != 1:
        raise ValueError('F must be monic in y')
    n = sympy.degree(F, x)
    _list_of_fs = fgetter(F)
    f0 = F.subs(x, 0)
    p, q = pqgetter(f0)
    q1, p1 = lemma_1(_list_of_fs[1], p, q)
    _list_of_ps = [p, p1]
    _list_of_qs = [q, q1]
    _list_of_ms = [0]
    for i in range(2,n+1):
        ri = 0
        for j in range(1,i):
            ri = ri + _list_of_qs[i-j]*_list_of_ps[j]
        mi = _list_of_fs[i] - ri
        _list_of_ms.append(mi)
        qi, pi = lemma_1(mi, p, q)
        _list_of_ps.append(sympy.Poly(pi,x,y))
        _list_of_qs.append(sympy.Poly(qi,x,y))
    P = 0
    Q = 0
    for k in range(0,n+1):
        Q = Q + sympy.Poly(x**k, x, y)*_list_of_qs[k]
    for k in range(0,n+1):
        P = P + sympy.Poly(x**k, x, y)*_list_of_ps[k]
        if P*Q == F:
            break
    print('list of fs: ' + str(_list_of_fs))
    print('list of ps: ' + str(_list_of_ps))
    print('list of qs: ' + str(_list_of_qs))
    print('list of ms: ' + str(_list_of_ms))
    return P, Q

In [10]:
def Hensels_lemma_2(F, n):
    x = sympy.Symbol('x')
    y = sympy.Symbol('y')

    if sympy.LC(F, y) != 1:
        raise ValueError('F must be monic in y')

    list_of_ps = [0 for k in range(0,n+1)]
    list_of_qs = [0 for k in range(0,n+1)]

    degree_of_F_x = sympy.degree(F, x)

    list_of_fs = [0 for k in range(0,n+1)]
    for k in range(0, degree_of_F_x+1):
        try:
            list_of_fs[k] = fgetter(F)[k]
        except:
            pass


    f0 = F.subs(x, 0)
    p, q = pqgetter(f0)
    q1, p1 = lemma_1(list_of_fs[1], p, q)
    
    list_of_ps[0] = p
    list_of_ps[1] = p1
    list_of_qs[0] = q
    list_of_qs[1] = q1

    list_of_ms = [0]
    for i in range(2,n+1):
        ri = 0
        for j in range(1,i):
            ri = ri + list_of_qs[i-j]*list_of_ps[j]
        mi = list_of_fs[i] - ri
        list_of_ms.append(mi)
        qi, pi = lemma_1(mi, p, q)
        list_of_ps[i] = sympy.Poly(pi,x,y)
        list_of_qs[i] = sympy.Poly(qi,x,y)
    P = 0
    Q = 0

    for k in range(0,n+1):
        Q = Q + sympy.Poly(x**k, x, y)*list_of_qs[k]
        P = P + sympy.Poly(x**k, x, y)*list_of_ps[k]
    
    print('list of fs: ' + str(list_of_fs))
    print('list of ps: ' + str(list_of_ps))
    print('list of qs: ' + str(list_of_qs))
    print('list of ms: ' + str(list_of_ms))

    return P, Q

In [11]:
x = sympy.Symbol('x')
y = sympy.Symbol('y')
F = sympy.Poly(y**3 + y + x*(y**2 + y) + y*x**2)
P, Q = HenselsLemma(F)
print(F == P*Q)
print('P: ' + str(P))
print('Q: ' + str(Q))
print('F: '+ str(F.factor()))
print('P*Q: '+ str(sympy.Poly(P*Q)))

PolynomialError: Poly(2*y, y, domain='ZZ') contains an element of the generators set

In [28]:
G = sympy.Poly(y**5 + y + x*(y + 2) - (y**2 + 2*y + 1)*x**2)
P, Q = HenselsLemma(G)
print(G == P*Q)
print('P: ' + str(P))
print('Q: ' + str(Q))
print('G: '+ str(G.factor()))
print('P*Q: '+ str(sympy.Poly(P*Q)))

list of fs: [Poly(y**5 + y, y, domain='ZZ'), Poly(1.0*y + 2.0, y, domain='RR'), Poly(-1.0*y**2 - 2.0*y - 1.0, y, domain='RR')]
list of ps: [y, Poly(2.0, y, domain='RR'), Poly(-3.0, x, y, domain='RR')]
list of qs: [y**4 + 1, Poly(-2.0*y**3 + 1.0, y, domain='RR'), Poly(3.0*y**3 + 4.0*y**2 - 1.0*y - 2.0, x, y, domain='RR')]
list of ms: [0, Poly(4.0*y**3 - 1.0*y**2 - 2.0*y - 3.0, y, domain='RR')]
False
P: Poly(-3.0*x**2 + 2.0*x + 1.0*y, x, y, domain='RR')
Q: Poly(3.0*x**2*y**3 + 4.0*x**2*y**2 - 1.0*x**2*y - 2.0*x**2 - 2.0*x*y**3 + 1.0*x + 1.0*y**4 + 1.0, x, y, domain='RR')
G: -x**2*y**2 - 2*x**2*y - x**2 + x*y + 2*x + y**5 + y
P*Q: Poly(-9.0*x**4*y**3 - 12.0*x**4*y**2 + 3.0*x**4*y + 6.0*x**4 + 12.0*x**3*y**3 + 8.0*x**3*y**2 - 2.0*x**3*y - 7.0*x**3 - 1.0*x**2*y**2 - 2.0*x**2*y - 1.0*x**2 + 1.0*x*y + 2.0*x + 1.0*y**5 + 1.0*y, x, y, domain='RR')
